In [1]:
import urllib3
import requests
from bs4 import BeautifulSoup
import json
import regex
import helper
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
def connectionError(link, data=""):
    """
    Return requests.get(link) with post request and test website issues

    :param link: URL
    :param data: data to give to the form

    :return: requests.get(link)
    """
    try:
        urllib3.disable_warnings()
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'}
        if data!= "":
            res = requests.post(link, data=data, headers=headers,verify=False)
        else:
            res = requests.get(link, allow_redirects=False,stream=True,verify=False)
        if res.status_code != 200:
            print('Server Error: ' + str(res.status_code) + '\n' + 'For url:' + link)
            #raise Exception('Server Error: ' + str(res.status_code) + '\n' + 'For url:' + link)
            # sys.exit(1)
        return res
    except requests.exceptions.RequestException as error:
        print("Can't connect: {} - Eror: {}".format(link,error))
        return

In [3]:
# link = "http://data.gramene.org/v60/genes?q=Os08g0494375"
# link = "http://data.gramene.org/v60/genes?_id=Os08g0494375"
# link = "http://expression.ic4r.org/api/fpkm?gene=Os01g0271700"
#link = "https://www.ebi.ac.uk/gxa/genes/os11g0559200"
# link = "https://urgi.versailles.inra.fr/wheatis-srv/results/facet?term=TRAES3BF001000010CFD&start=1&length=10"
#link = "https://www.ebi.ac.uk/gxa/genes/Os11g0116500"
link = "https://www.arabidopsis.org/servlets/TairObject?name=AT2G03340&type=locus"
#result = connectionError(link)
#result = requests.get(link, allow_redirects=False,stream=True,verify=False)

In [3]:
# options = Options()
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')  # Last I checked this was necessary.
# driver = webdriver.Chrome('/Users/mac/Downloads/chromedriver',options= options)
# driver.get(link)

In [4]:
# download_button = driver.find_elements_by_xpath("//button[@class=.btn.btn-sm.btn-default]")
# download_button.click()

In [5]:
# data=BeautifulSoup(driver.page_source, 'lxml')
# a = driver.find_element_by_css_selector('.btn.btn-sm.btn-default').click()

In [4]:
result = connectionError(link)
soup = BeautifulSoup(result.content,"lxml")
soup

<!-- Copyright (c) 2000-2010 Carnegie Institution for Science.  All rights reserved. --><!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="SEVsCs5SRo40Yv69trkf5mDpaKLbU1zI-D_tfu6LMvU" name="google-site-verification"/>
<title>AT2G03340(WRKY3)</title>
<link href="/favicon.ico" rel="Shortcut Icon"/>
<link href="/css/main.css" rel="stylesheet" type="text/css"/>
<link href="/css/search.css" rel="stylesheet" type="text/css"/>
<link href="/css/base.css?v=20181108" rel="stylesheet" type="text/css"/>
<script src="/js/prototype.js" type="text/javascript"></script>
<script src="/js/global.js?20091013141700" type="text/javascript"></script>
<script src="/js/menu.js" type="text/javascript"></script>
<script src="/js/footerNew.js" type="text/javascript"></script>
<script src="/js/init.js" type="text/javascript"></script>
<!--  script src='htt

In [5]:
ret = BeautifulSoup(result.content, "lxml")

In [6]:
database_description = helper.fetch_description("arabidpsis")

In [7]:
headers = []
for header in database_description[0].find_all("header"):
    headers.append(header.text)

In [8]:
print(database_description[0].findAll("data_struct")[0]["identifier"])

In [9]:
if database_description[0].findAll("data_struct")[0]["identifier"] != "":
    data = ret.find_all(database_description[0].find_all("data_struct")[0]["indicator"],
                        {database_description[0].find_all("data_struct")[0]["identifier"]:
                        database_description[0].find_all("data_struct")[0]["identification_string"]})
else:
    print('abc')
    data = ret.find_all(database_description[0].findAll("data_struct")[0]["indicator"])

abc


In [10]:
data = ret.find_all('table')

In [15]:
data

[<table border="0" cellspacing="0" width="100%">
 <tr>
 <td colspan="2" nowrap=""><b><font size="+1">Locus: AT2G03340</font><font color="red" size="+1"> </font></b></td>
 <script type="text/javascript">
 	function clickNewOnPageBtn(h) {
 	    var url = location.href;
 	    location.href = "#"+h;
 	    //history.replaceState(null,null,url);
 	}
 	</script>
 <td align="left" class="sm">
 <div class="popover__wrapper">
 <button onclick="clickNewOnPageBtn('GeneFamiliesBand')" style="color: black; background-color: #cccc99;">What's new on this page
 				</button>
 <div class="push popover__content">
 <img src="/images/GeneFamilyBandOnLocus.png"/>
 </div>
 </div>
 </td>
 <td align="right" class="sm">
 <!-- vet TAIR-2499 -->
 <form action="/servlets/updater" method="get">
 <input name="type" type="hidden" value="notepad"/> <input name="update_action" type="hidden" value="add"/> <input name="tair_object_id" type="hidden" value="2063835"/> <input name="accession" type="hidden" value="Locus:2063

In [11]:
if data != []:
    reg = regex.compile(database_description[0].findAll(
        "prettify")[0].text, regex.MULTILINE)
    replaceBy = database_description[0].findAll(
        "prettify")[0]['replaceBy']
    for dataLine in data[1].findAll(database_description[0].findAll("data_struct")[0]["line_separator"]):
        #print(dataLine)
        dict_ = {}
        i = 0
        for dataCell in dataLine.findAll(
                database_description[0].findAll("data_struct")[0]["cell_separator"]):
            dataFormat = regex.sub(replaceBy + '+', ' ', dataCell.text)
            if (i < len(headers)):
                dict_[headers[i]] = dataFormat
            i += 1
        if dict_ == {}:
            continue
        dict_.pop("", None)
        print(dict_)

{'Representative Gene Model': ' ', 'Gene Model Type': ' AT2G03340.1 '}
{'Representative Gene Model': ' ', 'Gene Model Type': 'protein_coding'}
{'Representative Gene Model': ' ', 'Gene Model Type': ' WRKY DNA-BINDING PROTEIN 3, WRKY3 ', 'Other names': 'WRKY DNA-BINDING PROTEIN 3, WRKY3'}
{'Representative Gene Model': 'WRKY DNA-BINDING PROTEIN 3, WRKY3'}
{'Representative Gene Model': ' ', 'Gene Model Type': 'Encodes WRKY DNA-binding protein 3 (WRKY3).'}
{'Representative Gene Model': ' ', 'Gene Model Type': ' Center on AT2G03340 | Full-screen view '}
{'Representative Gene Model': ' ', 'Gene Model Type': ' category relationship type keyword GO Biological Process involved in regulation of transcription, DNA-templated GO Cellular Component located in nucleus GO Molecular Function has DNA-binding transcription factor activity, sequence-specific DNA binding Growth and Developmental Stages expressed during LP.02 two leaves visible stage, LP.04 four leaves visible stage, LP.06 six leaves visible

In [26]:
data[4]

<table border="0" cellpadding="1" width="100%">
<tr>
<th align="left" class="sm" valign="top">category</th>
<th class="sm"> </th>
<th align="left" class="sm" nowrap="" valign="top">relationship
						type <a href="javascript:void(0);" onclick="javascript:launchDefinitions(101036);" onmouseover="window.status=''"><img alt="(?)" border="0" height="12" src="/images/questionmark.gif" width="10"/></a></th>
<th class="sm" nowrap=""> </th>
<th align="left" class="sm" valign="top">keyword <a href="javascript:void(0);" onclick="javascript:launchDefinitions(623);" onmouseover="window.status=''"><img alt="(?)" border="0" height="12" src="/images/questionmark.gif" width="10"/></a></th>
</tr>
<tr>
<td align="left" class="sm" valign="top">GO Biological Process
					</td>
<td class="sm"> </td>
<td align="left" class="sm" valign="top">involved in
					</td>
<td class="sm"> </td>
<td align="left" class="sm" valign="top"><a href="/servlets/TairObject?type=keyword&amp;id=7461" target="_other">regulation 

In [32]:
if data != []:
    reg = regex.compile("\n>LOC_.*\n|\n|\r|\t", regex.MULTILINE)
    replaceBy = database_description[0].findAll(
        "prettify")[0]['replaceBy']

In [34]:
tmp = ret.find_all('tr')

In [36]:
len(tmp)

281

In [11]:
soup = BeautifulSoup(result.content,"lxml")

In [12]:
soup

<!-- Copyright (c) 2000-2010 Carnegie Institution for Science.  All rights reserved. --><!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="SEVsCs5SRo40Yv69trkf5mDpaKLbU1zI-D_tfu6LMvU" name="google-site-verification"/>
<title>AT2G03340(WRKY3)</title>
<link href="/favicon.ico" rel="Shortcut Icon"/>
<link href="/css/main.css" rel="stylesheet" type="text/css"/>
<link href="/css/search.css" rel="stylesheet" type="text/css"/>
<link href="/css/base.css?v=20181108" rel="stylesheet" type="text/css"/>
<script src="/js/prototype.js" type="text/javascript"></script>
<script src="/js/global.js?20091013141700" type="text/javascript"></script>
<script src="/js/menu.js" type="text/javascript"></script>
<script src="/js/footerNew.js" type="text/javascript"></script>
<script src="/js/init.js" type="text/javascript"></script>
<!--  script src='htt

In [14]:
link = "https://www.ebi.ac.uk/gxa/genes/Os11g0116500#differential"
result = connectionError(link)
soup = BeautifulSoup(result.content,"lxml")
soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Search results &lt; Expression Atlas &lt; EMBL-EBI</title>
<meta content="EMBL-EBI Expression Atlas, an open public repository of gene expression pattern data under different biological conditions" name="description"/>
<meta content="expression atlas, gene expression, baseline expression, differential expression, functional genomics, public repository, repository, bioinformatics, europe, institute" name="keywords"/>
<meta content="EBI Gene Expression Team – https://www.ebi.ac.uk/about/people/irene-papatheodorou" name="author"/>
<meta content="true" name="HandheldFriendly"/>
<meta content="width" name="MobileOptimized"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="#70BDBD" name="theme-color"/> <!-- Android Chrome mobile browser tab color -->
<!-- Add information on the life cycle of this page -->
<meta content="Irene Papatheodorou &lt;irenep@ebi.ac.uk&gt;" name="ebi:owner"/>
<m